In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.17.4
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


## 1. 数据的读取与展示

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all),(x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[ :5000], x_train_all[5000: ]
y_valid, y_train = y_train_all[ :5000], y_train_all[5000: ]

### 1.1 数据归一化

In [3]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1, 28, 28)

## 2. 模型构建

### 2.1 构建架构

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# reason for spare:  y->index.   y->one_hot->[ ] 将索引转换为热独向量
model.compile(
    loss="sparse_categorical_crossentropy", # 如果已经是热独形式，则用                                                     # categorical_crossentropy函数
                                            # 使用sgd时会出现Nan的情况
              optimizer ="Adam",               
              metrics = ["accuracy"])                      # 关心的参数

### 2.2 查看模型架构

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


## 3.【重点】训练模型（保存模型参数）

In [6]:
logdir = "./graph_def_and_weights"
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, "fashion_mnist_weights.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir, profile_batch=100000000),     # 这里没有设置profile_batch参数时会出问题
    keras.callbacks.ModelCheckpoint(output_model_file, 
                                    save_best_only=True, # False则保存训练到最                                                          # 后的模型的参数
                                    save_weights_only=True), # 只保存参数不保存                                                              # 模型
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]
history = model.fit(x_train_scaled, y_train, epochs=10, 
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)     # 每次遍历之后都会使用一次验证集验证准确度

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 94us/sample - loss: 0.4590 - accuracy: 0.8324 - val_loss: 0.3681 - val_accuracy: 0.8656
Epoch 2/10
55000/55000 [==============================] - 5s 83us/sample - loss: 0.3530 - accuracy: 0.8696 - val_loss: 0.3481 - val_accuracy: 0.8782
Epoch 3/10
55000/55000 [==============================] - 4s 77us/sample - loss: 0.3152 - accuracy: 0.8820 - val_loss: 0.3344 - val_accuracy: 0.8774
Epoch 4/10
55000/55000 [==============================] - 4s 75us/sample - loss: 0.2943 - accuracy: 0.8904 - val_loss: 0.3439 - val_accuracy: 0.8774
Epoch 5/10
55000/55000 [==============================] - 4s 79us/sample - loss: 0.2757 - accuracy: 0.8965 - val_loss: 0.3045 - val_accuracy: 0.8920
Epoch 6/10
55000/55000 [==============================] - 4s 81us/sample - loss: 0.2558 - accuracy: 0.9047 - val_loss: 0.3102 - val_accuracy: 0.8914
Epoch 7/10
55000/55000 [==============================] -

## 4. 测试数据

In [7]:
model.evaluate(x_test_scaled, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3594793825984001, 0.8817]

## 5. 【重点】加载权重

### 5.1 重新设置模型

In [11]:
load_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# reason for spare:  y->index.   y->one_hot->[ ] 将索引转换为热独向量
load_model.compile(
    loss="sparse_categorical_crossentropy",                              
    optimizer ="Adam",               
    metrics = ["accuracy"])                      

### 5.2 加载参数

In [17]:
logdir = "./graph_def_and_weights"
output_model_file = os.path.join(logdir, "fashion_mnist_weights.h5")
load_model.load_weights(output_model_file)

In [18]:
load_model.evaluate(x_test_scaled, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.340694984793663, 0.878]

## 6. 另一种保存模型参数的方式

In [19]:
load_model.save_weights(os.path.join(logdir, "fashion_mnist_weights_2.h5"))